In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q opencv-contrib-python
!pip install -q controlnet_aux
!pip install flask_cors
!pip install pyngrok
!pip install flask
!pip install -q diffusers==0.14.0 transformers xformers git+https://github.com/huggingface/accelerate.git

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2OoJ32wvAsWUSA8HozSw8FpJ6kH_2xZE3Jntt4pPy1eFFQ1PR")
public = ngrok.connect(5000).public_url
print(public)

https://ba74-35-223-11-43.ngrok-free.app


In [ ]:
from io import BytesIO
import os
import base64
from PIL import Image
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from werkzeug.utils import secure_filename
# from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
from controlnet_aux import HEDdetector
from diffusers.utils import load_image
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
from controlnet_aux import HEDdetector
from diffusers.utils import load_image
# from flask_ngrok import run_with_ngrok
app = Flask(__name__)
# run_with_ngrok(app)

# Define the allowed file extensions
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}

# Set the upload folder and allow the allowed extensions
app.config['UPLOAD_FOLDER'] = '/content/drive/MyDrive/Marina/uploads/gg.png'
app.config['Output_image'] = '/content/drive/MyDrive/Marina/Output_img/generated.png'
app.config['ALLOWED_EXTENSIONS'] = ALLOWED_EXTENSIONS


@app.route('/api/hello', methods=['GET'])
def hello():
    return "hellow"

# Check if the uploaded file has allowed extension
def gen(image_pth, prompt):
  hed = HEDdetector.from_pretrained('lllyasviel/ControlNet')
  image = load_image(image_pth)
  image = hed(image, scribble=True)
  controlnet = ControlNetModel.from_pretrained(
      "lllyasviel/sd-controlnet-scribble", torch_dtype=torch.float16
  )
  pipe = StableDiffusionControlNetPipeline.from_pretrained(
      "runwayml/stable-diffusion-v1-5", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16
  )
  pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
  # Remove if you do not have xformers installed
  # see https://huggingface.co/docs/diffusers/v0.13.0/en/optimization/xformers#installing-xformers
  # for installation instructions
  pipe.enable_xformers_memory_efficient_attention()
  pipe.enable_model_cpu_offload()
  # image = pipe("house", image, num_inference_steps=20).images[0]
  image = pipe(prompt, image, num_inference_steps=20).images[0]
  image.save('/content/drive/MyDrive/Marina/Output_img/generated.png')
  output_pth = '/content/drive/MyDrive/Marina/Output_img/generated.png'

  return output_pth

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


@app.route('/api/process_image', methods=['POST'])
def process_image():
    response = jsonify({'success': False})
    # Check if a file is uploaded
    if 'image' not in request.form:
        return jsonify({'success': False, 'error': 'No file part'})

    # Get the prompt
    prompt = request.form['prompt']
    # Get the image
    file = request.form['image']
    file = file.split(",")[1] # Remove the "data:image/png;base64," part
    try:
        image_bytes = BytesIO(base64.b64decode(file))
        img = Image.open(image_bytes)
    except Exception as e:
        return jsonify({'success': False, 'error': 'Error while converting file type to PIL. {}'.format(e)})

    # Save the file in upload folder
    try:
        img.save(app.config['UPLOAD_FOLDER'], "PNG")
    except:
        return jsonify({'success': False, 'error': 'Error while saving file'})
    try:
      finalPath = gen(app.config['UPLOAD_FOLDER'], prompt)
      app.config['Output_image'] = finalPath
    except Exception as e:
      print("There errors in image generation. {}".format(e))
      return jsonify({'success': False, 'error': "There errors in image generation. {}".format(e)})
    # Return the uploaded image
    try:
        return jsonify({'success': True, 'error': ''})
    except:
        return jsonify({'success': False, 'error': 'Error while sending file'})

@app.route('/image')
def download_image():
    filename = '/content/drive/MyDrive/Marina/Output_img/generated.png'  # Replace with the name of your image file
    return send_file(filename, as_attachment=True)

if __name__ == '__main__':
    CORS(app, origins="http://localhost:3000")
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Apr/2023 07:18:18] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Apr/2023 07:18:27] "GET /api/hello HTTP/1.1" 200 -


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/20 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [23/Apr/2023 07:20:30] "POST /api/process_image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Apr/2023 07:20:39] "GET /api/image HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Apr/2023 07:20:39] "GET /favicon.ico HTTP/1.1" 404 -
